In [ ]:
import requests
import psycopg2
import pandas as pd
from io import StringIO
from datetime import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Email Notification Function
def send_email(subject, message, sender_email, sender_password, receiver_email):
    """Send an email notification."""
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'plain'))
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()
        print("Email notification sent successfully.")
    except Exception as e:
        print(f"Failed to send email notification: {e}")

def fetch_tickers_and_latest_dates(conn_params):
    """Fetch tickers and their latest date from the database."""
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    cursor.execute("SELECT ticker, MAX(date) FROM usa_stocks_new GROUP BY ticker")
    results = cursor.fetchall()
    cursor.close()
    conn.close()
    return {ticker: pd.to_datetime(latest_date) for ticker, latest_date in results}

def fetch_bulk_eod(api_token):
    """Fetch EOD data."""
    url = f"https://eodhd.com/api/eod-bulk-last-day/US?api_token={api_token}&fmt=json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error fetching data: {response.status_code}")

def insert_data_to_db(data, conn_params, latest_dates):
    """Insert data into the database with checks for duplicates and date gaps."""
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()

    df = pd.DataFrame(data)
    df = df[['code', 'date', 'close', 'volume']]
    df.columns = ['ticker', 'date', 'price', 'volume']
    df['date'] = pd.to_datetime(df['date'])
    df = df[df.apply(lambda x: x['date'] > latest_dates.get(x['ticker'], datetime.min), axis=1)]

    if df.empty:
        print("No new data to update. Database is up to date.")
        return

    buffer = StringIO()
    df.to_csv(buffer, index=False, header=False)
    buffer.seek(0)
    cursor.copy_from(buffer, 'usa_stocks_new', sep=",", columns=('ticker', 'date', 'price', 'volume'))
    conn.commit()
    cursor.close()
    conn.close()

    print(f"Updated data for {', '.join(sorted(set(df['ticker'])))}.")
    return df

api_token = '648bce67d09e19.82875075'
conn_params = {
    'dbname': 'Datarame_dev',
    'user': 'doadmin',
    'password': 'AVNS_AmS_EzaaZTG7fU7ia5g',
    'host': 'db-datarame-nyc1-71861-do-user-14255173-0.b.db.ondigitalocean.com',
    'port': '25060',
}

sender_email = "itsquandala@gmail.com"
sender_password = "nqtj eqkb eufv wcnd"  # Securely fetch this
receiver_email = "s.apanavicius@gmail.com"

# Main script execution
try:
    ticker_latest_dates = fetch_tickers_and_latest_dates(conn_params)
    bulk_eod_data = fetch_bulk_eod(api_token)
    filtered_eod_data = [data for data in bulk_eod_data if data['code'] in ticker_latest_dates]
    updated_data = insert_data_to_db(filtered_eod_data, conn_params, ticker_latest_dates)
    if updated_data is not None:
        send_email("Datarame_dev Database Update Success",
                   f"Updated data for {', '.join(sorted(set(updated_data['ticker'])))}.",
                   sender_email, sender_password, receiver_email)
    else:
        send_email("Datarame_dev Database Update", "No new data to update. Datarame_dev Database is up to date.",
                   sender_email, sender_password, receiver_email)
except Exception as e:
    error_message = f"An error occurred: {str(e)}"
    print(error_message)
    send_email("Datarame_dev Database Update Failed", error_message, sender_email, sender_password, receiver_email)